## DBMS (Database Management System):
It is a software used to define, manipulate, retrieve, store and manage data in database.

- Defining rules to validate and manipulate data.
- Interacting with databases, applications and end users.
- Retrieving, storing and analyzing data.
- Updating data.

##  RDBMS (Relational Database Management System):

- Is a database management system that is based on the relational model.
- Components: Table, Record/Tuple/Row, Field, and Column/Attribute.
- An RDB has the ability to establish links—or relationships–between information by joining tables.
- Makes it easy to understand and gain insights about the relationship between various data points.

Different types of RDBMS's:
- **MySQL**
- **PostgreSQL**
- **Microsoft SQL Server**
- **Oracle Database**

# How to work with psycopg2

In [ ]:
import psycopg2

Connection = psycopg2.connect(database="northwind", user="postgres",
                            password="admin", host="localhost")
                            
curser = Connection.cursor()
curser.execute("select * from products")
print(type(curser))

for row in curser:
    print(row)

In [ ]:
query = '''select product_id, product_name from products
where discontinued = 0
order by product_name
limit 5'''
curser.execute(query)

In [ ]:
for row in curser:
    print(row)

Write tables and read from tables

## **Connect function:**

In [ ]:
def connect():
    return psycopg2.connect(database="northwind", user="postgres",
                            password="admin", host="localhost")

## **Creating Table with psycopg2:**

- A function for creating two tables, namely, 'users', and 'contacts', to use psycppg2:

In [ ]:
def create_table():
    commands = (
        """
        CREATE TABLE IF NOT EXISTS users (
            user_id INT GENERATED ALWAYS AS IDENTITY,
            first_name VARCHAR(255) NOT NULL,
            last_name VARCHAR(255) NOT NULL,
            PRIMARY KEY (user_id)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS contacts (
            contact_id INT GENERATED ALWAYS AS IDENTITY,
            user_id INT,
            contact_name VARCHAR(255) NOT NULL,
            phone VARCHAR(20) NOT NULL,
            email VARCHAR(100) NOT NULL,
            PRIMARY KEY (contact_id),
            CONSTRAINT fk_user
                FOREIGN KEY(user_id)
                REFERENCES users(user_id)
        )
        """
    )
    conn = None
    try:
        conn = connect()
        curser = conn.cursor()
        
        for command in commands:
            curser.execute(command)
        curser.close()
        conn.commit()
        
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
create_table()

In [ ]:
class User:
    def __init__(self, first_name, last_name, user_id) -> None:
        self.first_name = first_name
        self.last_name = last_name
        self.user_id = user_id
    
    def __repr__(self) -> str:
        return "<User {}>".format(self.user_id)

    def save_to_db(self):
        with connect() as connection:
            with connection.cursor() as curser:
                curser.execute("INSERT INTO users (first_name, last_name) VALUES (%s, %s)",
                                (self.first_name, self.last_name)
                            )
    @classmethod
    def load_from_db_by_id(cls, user_id):
        with connect() as connection:
            with connection.cursor() as curser:
                curser.execute("SELECT * from users where user_id=%d"% (user_id))
                user_data = curser.fetchone()
                return cls(first_name=user_data[1], last_name=user_data[2],
                            user_id=user_data[0])

In [ ]:
class Contact:
    def __init__(self, user_id, contact_name, phone, email, connect_id) -> None:
        self.user_id = user_id
        self.contact_name = contact_name
        self.phone = phone
        self.email = email
        self.contact_id = connect_id
    
    def __repr__(self) -> str:
        return "<Contact {}>".format(self.contact_id)

    def save_to_db(self):
        with connect() as connection:
            with connection.cursor() as curser:
                curser.execute('''INSERT INTO contacts (user_id, contact_name, phone, email) VALUES 
                                ((SELECT user_id FROM users where user_id=%s), %s, %s, %s)''',
                                (self.user_id, self.contact_name, self.phone, self.email)
                            )
    
    def print_data(self):
        print("User id: {}, Contact Name: {}, Phone: {}, Email: {}".format(
            self.user_id, self.contact_name, self.phone, self.email))

    @classmethod
    def load_from_db_by_email(cls, email):
        with connect() as connection:
            with connection.cursor() as curser:
                curser.execute("SELECT * from contacts where email=%s", (email,))
                contact_data = curser.fetchone()
                return cls(user_id=contact_data[1], contact_name=contact_data[2],
                            phone=contact_data[3], email=contact_data[4], connect_id=contact_data[0])

In [ ]:
my_user = User("Mammad", "Hoseini", None)
my_user.save_to_db()

In [ ]:
load_user = User.load_from_db_by_id(5)
print(load_user.first_name)
print(load_user.last_name)

In [ ]:
my_contact = Contact(5, "mahdavi", "25262", "aij@example.com", None)

In [ ]:
my_contact.save_to_db()

In [ ]:
load_contact = Contact.load_from_db_by_email("aij@example.com")

In [ ]:
print(load_contact.contact_id)
print(load_contact.contact_name)
print(load_contact.email)
print(load_contact.phone)
print(load_contact.user_id)

### Query